In [22]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [39]:
import pandas as pd
import numpy as np
import torch
import src.preprocessing as pre
import plotly.graph_objects as go
import src.utils as utils
from torch.utils.data import DataLoader
import src.table as table
import src.graphs as graphs

In [3]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')

In [24]:
model = utils.create_model(train,num_epochs=200)

Epoch [1/200], Loss: 4389163.9872
Epoch [2/200], Loss: 3853881.9091
Epoch [3/200], Loss: 3187569.8864
Epoch [4/200], Loss: 3013161.8580
Epoch [5/200], Loss: 2838465.7621
Epoch [6/200], Loss: 2655010.5142
Epoch [7/200], Loss: 2522540.3324
Epoch [8/200], Loss: 2449598.0639
Epoch [9/200], Loss: 2410802.5135
Epoch [10/200], Loss: 2382798.2933
Epoch [11/200], Loss: 2357347.3530
Epoch [12/200], Loss: 2325245.5838
Epoch [13/200], Loss: 2318584.5611
Epoch [14/200], Loss: 2273339.2209
Epoch [15/200], Loss: 2252881.2202
Epoch [16/200], Loss: 2234160.8416
Epoch [17/200], Loss: 2202279.1065
Epoch [18/200], Loss: 2172760.4943
Epoch [19/200], Loss: 2150082.2173
Epoch [20/200], Loss: 2118961.8736
Epoch [21/200], Loss: 2094525.4105
Epoch [22/200], Loss: 2061340.1151
Epoch [23/200], Loss: 2019425.1385
Epoch [24/200], Loss: 1990692.1314
Epoch [25/200], Loss: 1951575.4016
Epoch [26/200], Loss: 1918591.1783
Epoch [27/200], Loss: 1879810.6655
Epoch [28/200], Loss: 1843192.8274
Epoch [29/200], Loss: 1801209

In [40]:
torch.save(model.state_dict(),'model1.pth')

In [25]:
preds = utils.predict_gridded_harvest(model,test)

In [26]:
actuals = test.Y.detach().numpy()

In [27]:
production_plan = pre.decode(meta,mappings).reset_index(drop=True)

In [28]:
idx_dict = table.create_indices_dict(production_plan)

In [29]:
transplant_weeks = production_plan.WeekTransplanted.to_numpy()
year_preds, year_actuals = table.season_shift(transplant_weeks,preds,actuals)
year_preds.shape,year_actuals.shape

((139, 20, 51), (139, 51))

In [30]:
fig1 = graphs.graph_harvest(preds,actuals)
fig2 = graphs.graph_harvest(year_preds,year_actuals)

In [31]:
fig3 = graphs.graph_harvest_cumsum(preds,actuals)

In [32]:
types_i_want_lol = ['Pera Roja','Uva Amarilla']

In [33]:
preds_sum,actuals_sum = table.filter_preds(preds,actuals,'Type',types_i_want_lol,idx_dict)

In [34]:
fig4 = graphs.graph_harvest_stacked(preds_sum,actuals_sum,types_i_want_lol)

In [35]:
fig1.show()

In [36]:
fig2.show()

In [37]:
fig3.show()

In [21]:
preds.shape,actuals.shape

((139, 20, 20), (139, 20))

In [38]:
fig4.show()
